In [3]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 
headers = {'User Agent':'Mozilla/5.0'}

players = ['paolo-banchero', 'jabari-smith-2', 'jonathan-davis-3', 'ej-liddell', 'chet-holmgren', 'jaden-ivey', 
           'bennedict-mathurin', 'blake-wesley', 'bryce-mcgowens', 'max-christie', 'patrick-baldwinjr', 
           'tyty-washingtonjr', 'jalen-duren',  'julian-strawther', 'kendall-brown', 'caleb-houstan', 
           'aminu-mohammed', 'kennedy-chandler', 'keegan-murray', 'wendell-moorejr', 'dereon-seabron',
           'ochai-agbaji', 'mark-williams-7', 'caleb-love', 'trayce-jackson-davis', 'terrence-shannonjr',
           'jamaree-bouyea', 'jaime-jaquezjr', 'johnny-juzang', 'jahvon-quinerly', 'matthew-mayer', 
           'trevor-keels', 'orlando-robinson', 'taevion-kinsey', 'anthony-duruji', 'keon-ellis',
           'julian-champagnie', 'michael-devoe', 'andrew-nembhard', 'oscar-tshiebwe','iverson-molinar', 
           'jahmir-young', 'tyson-etienne', 'alex-barcello', 'max-abmas', 'kevin-obanor','brady-manek', 
           'izaiah-brockington', 'moussa-diabate', 'jd-davison', 'aj-griffin', 'tari-eason','jabari-walker']

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2022')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2022'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append(22)
        year_list.append(.85)

age_list[1] = 19.1
year_list[1] = 1.07
age_list[2] = 20.3
year_list[2] = .96
age_list[3] = 21.6
year_list[3] = .83
table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

per40 = 40/table['MP']

table["Player Grade"] = ((table['PTS']*per40) + (table['TRB']*1.5*per40) + (table['AST']*2*per40) +
(table['BLK']*3*per40) + (table['STL']*3*per40) + (table['3P']*5*per40)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.25
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['Age']
table

,Name,Season,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,,Player Grade
1,Jabari Smith,2021-22,Auburn,7.27,20,27.5,.438,2.1,5.2,0.404,0.800,6.5,2.0,1.3,1.1,1.7,1.9,15.2,,113.4
2,Chet Holmgren,2021-22,Gonzaga,3.19,17,25.3,.618,1.1,2.8,0.396,0.738,8.5,2.1,0.5,3.4,1.9,2.1,13.6,,105.4
3,Jonathan Davis,2021-22,Wisconsin,9.17,16,34.8,.438,1.7,4.6,0.365,0.802,7.4,2.6,1.4,0.7,2.3,1.6,22.3,,95.8
4,Paolo Banchero,2021-22,Duke,4.21,19,30.8,.510,1.1,3.1,0.339,0.755,7.9,2.7,1.2,0.9,2.4,1.8,17.9,,94.2
5,Aj Griffin,2021-22,Duke,4.21,19,18.6,.509,1.4,3.1,0.458,0.750,3.2,1.1,0.4,0.5,0.5,1.1,8.2,,92.1
6,Bennedict Mathurin,2021-22,Arizona,5.01,18,31.4,.467,2.3,5.9,0.393,0.770,6.3,2.3,0.9,0.3,2.0,1.9,17.4,,90.2
7,Kennedy Chandler,2021-22,Tennessee,8.65,17,30.1,.442,1.2,3.8,0.323,0.604,3.1,4.9,2.3,0.4,2.6,2.0,13.3,,89.7
8,Jaden Ivey,2021-22,Purdue,5.08,18,29.3,.485,2.0,4.7,0.424,0.747,5.0,2.9,1.2,0.5,2.2,1.9,16.7,,89.4
9,Jalen Duren,2021-22,Memphis,5.84,15,25.5,.617,0.0,0.1,0.000,0.549,7.3,1.2,0.7,2.5,1.9,2.7,10.7,,87.5
10,Ej Liddell,2021-22,Ohio State,6.88,16,30.7,.512,1.4,3.6,0.397,0.732,7.3,2.7,0.4,2.8,2.9,2.5,19.4,,87.3
